In [10]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
    concatenate, Flatten, Lambda, Concatenate
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt


import tensorflow as tf
    
   
def load_class_ids(class_info_file_path):
    """
    Load class ids from class_info.pickle file
    """
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids


def load_embeddings(embeddings_file_path):
    """
    훈련된 텍스트 임베딩을 불러옴
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

def load_bounding_boxes(dataset_dir):
    """
    이미지와 그에 상응하는 바운딩 박스를 짝지어 딕셔너리로 만들어 출력
    """
    # 바운딩 박스 전체 경로
    bounding_boxes_path = os.path.join(dataset_dir, 'list_bbox_celeba_pure.csv')
    file_paths_path = os.path.join(dataset_dir, 'list_filenames.csv')

    # bounding_boxes.txt 와 images.txt 파일을 읽어옴
    df_bounding_boxes = pd.read_csv(bounding_boxes_path, header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, header=None)

    # 전체 이미지 파일 명이 순서대로 적힌 리스트를 만듬
    file_names = df_file_names[0].tolist() 

    # 파일 이름에 대응하는 바운딩 박스가 들어갈 딕셔너리를 만듬 (딕셔너리는 크기를 임의로 증가시킬수 있으므로 초기 사이즈는 아무렇게나)
    filename_boundingbox_dict = {}

    # 이미지 파일과 그에 해당하는 바운딩 박스를 딕셔너리로 만듬 (key = 이미지 파일 이름)
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][:].tolist()
        key = file_names[i][:-4] + '.jpg'
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict
'''
새 이미지가 크롭핑 되어있지 않기 크롭하기 위한 바운딩 박스 좌표 값이 파일에 주어지며,
그 파일을 토대로 이미지를 크로핑 한 후,
크로핑된 모든 이미지를 지정한 이미지 크기 (image_size) 값으로 바꾼다
'''
def get_img(img_path, bbox, image_size):
    """
    Load and resize image
    """
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        pass
    '''
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
    '''
    img = img.resize(image_size, PIL.Image.BILINEAR)
    return img


def load_dataset(filenames_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    """
    Load dataset
    """
    filenames = load_filenames(filenames_file_path)
    '''
    class_ids = load_class_ids(class_info_file_path)
    '''
    bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    # 각 이미지에 해당하는 바운딩 박스 딕셔너리를 추출하여 get_img 함수로 크로핑되고 같은 크기로 바뀐 이미지를 
    for index, filename in enumerate(filenames):
        bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{0}/images/{1}'.format(cub_dataset_dir, filename)
            img = get_img(img_name, bounding_box, image_size)
            '''
            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            '''
            embedding = all_embeddings[index, :]
            # X = 정제한 이미지 리스트
            X.append(np.array(img))
            '''
            # y = 정제한 이미지 인덱스
            y.append(class_ids[index])
            '''
            # embeddings = 
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, embeddings


def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c


def build_ca_model():
    """
    (1024,)의 텍스트 인코더 신경망의 출력을 입력으로 받고 (256,) 의 텐서를 출력
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model


def build_embedding_compressor_model():
    """
    입력 속성값 (40,) 을 (128,) 의 벡터로 확장하는 네트워크
    """
    input_layer = Input(shape=(40,))
    x = Dense(10)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model


def build_stage1_generator():
    """
    Stage-I 의 generator 
    *** 이 신경망 안에 CA 신경망과 생성기 신경망이 들어가 있다!!!! ***
    그러므로, 입력으로 텍스트 임베딩 출력 (1024,)과 잡음 변수(100,) 을 받는다
    """
    '''
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))
    gen_input = Concatenate(axis=1)([c, input_layer2])
    '''
    # 텍스트 조건부 변수를 잡음 변수와 접합(concatenation) -> cGAN

    input_layer = Input(shape=(40,))
    x = Dense(128 * 8 * 4 * 4, use_bias=False)(input_layer)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=input_layer, outputs=x)
    '''
    stage - I gen 은 입력된 문장의 임베딩을 바탕으로 (+잡음 변수) 이미지를 생성 함 
    '''
    return stage1_gen


def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    '''
    실제 이미지에 해당하는 압축된 임베딩을 입력
    '''
    input_layer2 = Input(shape=(4, 4, 10))

    '''
    입력 이미지와 압축 텍스트 임베딩을 합침
    '''
    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=x2)
    '''
    출력은 입력 이미지가 진짜인지 가짜인지에 관한 확률(sigmoid)을 출력
    '''
    return stage1_dis


def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(40,))
    input_layer3 = Input(shape=(4, 4, 10))

    x = gen_model(input_layer)

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer3], outputs=valid)
    return model


def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss


def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)


def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()


def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    with callback.as_default():
          tf.summary.scalar(name, loss, batch_no)
          callback.flush()








'''
Stage - I stackGAN 훈련
'''
if __name__ == '__main__':
    '''
    하이퍼파라미터(불변 파라미터) 지정
    '''
    data_dir = "/home/csle/Desktop/CelebA_dataset_reduce"
    train_dir = data_dir + "/train"
    test_dir = data_dir + "/test"
    image_size = 64
    batch_size = 128
    z_dim = 100
    stage1_generator_lr = 0.0002
    stage1_discriminator_lr = 0.0002
    stage1_lr_decay_step = 600
    epochs = 200
    condition_dim = 10

    embeddings_file_path_train = train_dir + "/attr_(embeddings).pickle"
    embeddings_file_path_test = test_dir + "/attr_(embeddings).pickle"

    filenames_file_path_train = train_dir + "/filenames.pickle"
    filenames_file_path_test = test_dir + "/filenames.pickle"

    '''
    class_info_file_path_train = train_dir + "/class_info.pickle"
    class_info_file_path_test = test_dir + "/class_info.pickle"
    '''

    cub_dataset_dir = data_dir + "/img_align_celeba"

    '''
    optimizer 정의
    '''
    dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

    """"
    dataset 로드하기
    """
    X_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                      cub_dataset_dir=cub_dataset_dir,
                                                      embeddings_file_path=embeddings_file_path_train,
                                                      image_size=(64, 64))

    X_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                   cub_dataset_dir=cub_dataset_dir,
                                                   embeddings_file_path=embeddings_file_path_test,
                                                   image_size=(64, 64))


embeddings:  (16000, 40)
Embeddings shape: (16000, 40)
embeddings:  (4000, 40)
Embeddings shape: (4000, 40)


In [11]:
#with tf.device('/gpu:0'): 

import tensorflow as tf 

config = tf.compat.v1.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.6 
session = tf.compat.v1.Session(config=config)

    
"""
신경망 빌드 & compile
"""
'''
ca_model = build_ca_model()
ca_model.compile(loss="binary_crossentropy", optimizer="adam")
'''
stage1_dis = build_stage1_discriminator()
stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

stage1_gen = build_stage1_generator()
stage1_gen.compile(loss="mse", optimizer=gen_optimizer)

embedding_compressor_model = build_embedding_compressor_model()
embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

'''
stage-I GAN 빌드& 컴파일
이때, stage-I 의 discriminator 는 훈련시키지 않고 stage-I generator 의 가중치만 업데이트
'''
adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                          optimizer=gen_optimizer, metrics=None)

"""
tensorboard.set_model(stage1_gen)
tensorboard.set_model(stage1_dis)
'''
tensorboard.set_model(ca_model)
'''
tensorboard.set_model(embedding_compressor_model)
"""
# Generate an array containing real and fake values
# Apply label smoothing as well
real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1
'''
매 epoch 마다 아래를 반복함
'''
for epoch in range(epochs):
    print("========================================")
    print("Epoch is:", epoch)
    print("Number of batches", int(X_train.shape[0] / batch_size))

    gen_losses = []
    dis_losses = []

    # Load data and train model
    number_of_batches = int(X_train.shape[0] / batch_size)
    for index in range(number_of_batches):
        print("Batch:{}".format(index+1))
        '''
        모델에 입력으로 들어갈 이미지와 텍스트 임베딩을 받아옴 (각 텍스트 임베딩은 각 이미지에 대응 됨)
        '''
        # 원래는 CA 의 출력이 stage-I 으로 들어가야 하지만 gen 안에 CA 가 있음
        z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
        # 배치 사이즈만큼 훈련(실제) 이미지를 추출
        image_batch = X_train[index * batch_size:(index + 1) * batch_size]
        # 추출한 이미지에 대응하는 임베딩을 추출
        embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
        # 이미지들을 정규화하여 값을 작게 만듬
        image_batch = (image_batch - 127.5) / 127.5

        # stage-I 의 gen 에서 텍스트 임베딩을 바탕으로 저 해상도 fake 이미지를 생성
        # 이때, 두 stage 는 랜덤하게 생성한 텍스트 임베딩을 나머지 입력으로 받음
        fake_images = stage1_gen.predict(embedding_batch, verbose=3)

        # stage-I dis 에 들어갈 압축 텍스트 임베딩을 랜덤하게 생성한 텍스트 임베딩 기반으로 생성
        compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
        compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
        compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))
        '''
        discriminator 에서 입력 이미지가 CNN 을 통과한 결과와 속성값 (40,) 을 concatenate 해 주기 위해서는,
        속성값 (40,) 를 compressor 네트워크를 통해 (128,) 로 확장하고,
        reshape, tile 함수를 이용해 CNN 결과 (4, 4, 512) 와 (4, 4, :) 부분을 맞추어 주어야 한다.
        '''

        """
        stage-I dis 를 훈련함
        """
        # 실제 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 1 (이미지가 진짜라는 의미) 로 하여,
        # dis 가 실제 이미지를 잘 분류 하게끔 훈련
        dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                  np.reshape(real_labels, (batch_size, 1)))
        # gen 이 생성한 가짜 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 0 (이미지가 가짜라는 의미) 로 하여,
        # dis 가 가짜 이미지를 잘 분류 하게끔 훈련
        dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                  np.reshape(fake_labels, (batch_size, 1)))
        # 실제 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 0 (이미지가 가짜라는 의미) 로 하여,
        # dis 가 실제 이미지를 잘 분류 하게끔 훈련
        dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                   np.reshape(fake_labels[1:], (batch_size-1, 1)))

        d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

        print("d_loss_real:{}".format(dis_loss_real))
        print("d_loss_fake:{}".format(dis_loss_fake))
        print("d_loss_wrong:{}".format(dis_loss_wrong))
        print("d_loss:{}".format(d_loss))

        """
        stage-I GAN 을 훈련함
        이때, stage-I 의 discriminator 는 훈련시키지 않고 stage-I generator 의 가중치만 업데이트
        """
        g_loss = adversarial_model.train_on_batch([embedding_batch, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
        print("g_loss:{}".format(g_loss))

        dis_losses.append(d_loss)
        gen_losses.append(g_loss)

    """
    각 epoch 마다 Tensorboard 에 loss 저장
    """
    writer = tf.summary.create_file_writer("logs/".format(time.time()))
    write_log(writer, 'discriminator_loss', np.mean(dis_losses), epoch)
    write_log(writer, 'generator_loss', np.mean(gen_losses[0]), epoch)
    '''
    tf.summary.scalar('discriminator_loss', np.mean(dis_losses), epoch)
    tf.summary.scalar('generator_loss', np.mean(gen_losses[0]), epoch)

    tf.summary.scalar('discriminator_loss', np.mean(dis_losses))
    tf.summary.scalar('generator_loss', np.mean(gen_losses[0]))
    summary_op = tf.summary.merge()
    summary_writer = tf.summary.FileWriter("logs/".format(time.time()))
    '''    
    # 매 두번의 epoch 마다 이미지 gen & 이미지 저장
    if epoch % 2 == 0:
        # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
        z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
        embedding_batch = embeddings_test[0:batch_size]
        fake_images = stage1_gen.predict_on_batch(embedding_batch)

        # Save images
        for i, img in enumerate(fake_images[:10]):
            save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i))

    if epoch % 50 == 0:
        # Save models
        stage1_gen.save_weights("weights/stage1_gen_epoch_{}.h5".format(epoch))
        stage1_dis.save_weights("weights/stage1_dis_epoch_{}.h5".format(epoch))

'''
이제 훈련된 stage-I 의 generator 와 discriminator 을 얻음 (+ embedding_compressor) 
'''

Epoch is: 0
Number of batches 125
Batch:1
d_loss_real:0.5674400329589844
d_loss_fake:3.93941330909729
d_loss_wrong:8.893132209777832
d_loss:3.4918563961982727
g_loss:0.7205632925033569
Batch:2
d_loss_real:0.4806445837020874
d_loss_fake:3.3400704860687256
d_loss_wrong:0.07563874870538712
d_loss:1.0942496005445719
g_loss:0.790033221244812
Batch:3
d_loss_real:4.4705915451049805
d_loss_fake:0.019957419484853745
d_loss_wrong:1.3215755224227905
d_loss:2.5706790080294013
g_loss:0.7175033092498779
Batch:4
d_loss_real:1.7763992547988892
d_loss_fake:0.9410151839256287
d_loss_wrong:1.3400355577468872
d_loss:1.4584623128175735
g_loss:0.8007746338844299
Batch:5
d_loss_real:2.490717649459839
d_loss_fake:0.0016522035002708435
d_loss_wrong:1.1598907709121704
d_loss:1.5357445683330297
g_loss:0.7416332364082336
Batch:6
d_loss_real:1.8573157787322998
d_loss_fake:0.406360924243927
d_loss_wrong:1.0868208408355713
d_loss:1.3019533306360245
g_loss:0.7571575045585632
Batch:7
d_loss_real:1.9605002403259277
d_l

g_loss:0.3256359100341797
Batch:39
d_loss_real:1.0105854272842407
d_loss_fake:0.001315642730332911
d_loss_wrong:0.8377246856689453
d_loss:0.7150527957419399
g_loss:0.32556819915771484
Batch:40
d_loss_real:1.0535260438919067
d_loss_fake:0.0013164648553356528
d_loss_wrong:0.7762072682380676
d_loss:0.7211439552193042
g_loss:0.32534152269363403
Batch:41
d_loss_real:1.0009241104125977
d_loss_fake:0.0014823715901002288
d_loss_wrong:0.8479294776916504
d_loss:0.7128150175267365
g_loss:0.32694923877716064
Batch:42
d_loss_real:1.0122570991516113
d_loss_fake:0.0014736742014065385
d_loss_wrong:0.8330556750297546
d_loss:0.714760886883596
g_loss:0.32527846097946167
Batch:43
d_loss_real:0.9823076725006104
d_loss_fake:0.001914338325150311
d_loss_wrong:0.8415480256080627
d_loss:0.7020194272336084
g_loss:0.3252413868904114
Batch:44
d_loss_real:1.0798574686050415
d_loss_fake:0.0016665365546941757
d_loss_wrong:0.7479703426361084
d_loss:0.7273379541002214
g_loss:0.3253178596496582
Batch:45
d_loss_real:0.98

g_loss:0.32529711723327637
Batch:91
d_loss_real:0.91948002576828
d_loss_fake:0.0013579782098531723
d_loss_wrong:0.7293640375137329
d_loss:0.6424205168150365
g_loss:0.3253640830516815
Batch:92
d_loss_real:0.9251675605773926
d_loss_fake:0.0011425771517679095
d_loss_wrong:0.7350002527236938
d_loss:0.6466194877575617
g_loss:0.3253057599067688
Batch:93
d_loss_real:0.9127361178398132
d_loss_fake:0.0011739622568711638
d_loss_wrong:0.7725497484207153
d_loss:0.6497989865893032
g_loss:0.32541149854660034
Batch:94
d_loss_real:0.9415727853775024
d_loss_fake:0.000544748967513442
d_loss_wrong:0.7279300093650818
d_loss:0.6529050822719
g_loss:0.32533982396125793
Batch:95
d_loss_real:0.9358800649642944
d_loss_fake:0.0011678000446408987
d_loss_wrong:0.7683361768722534
d_loss:0.6603160267113708
g_loss:0.32524192333221436
Batch:96
d_loss_real:0.9626937508583069
d_loss_fake:0.0011251483811065555
d_loss_wrong:0.7146021127700806
d_loss:0.6602786907169502
g_loss:0.32533442974090576
Batch:97
d_loss_real:0.9399

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Epoch is: 1
Number of batches 125
Batch:1
d_loss_real:0.9143885374069214
d_loss_fake:0.000812365033198148
d_loss_wrong:0.695991575717926
d_loss:0.6313952538912417
g_loss:0.3251771330833435
Batch:2
d_loss_real:0.9000163078308105
d_loss_fake:0.0007702287985011935
d_loss_wrong:0.7240316271781921
d_loss:0.6312086179095786
g_loss:0.32513007521629333
Batch:3
d_loss_real:0.8821622133255005
d_loss_fake:0.0009162353817373514
d_loss_wrong:0.7130714058876038
d_loss:0.6195780169800855
g_loss:0.3251776397228241
Batch:4
d_loss_real:0.9051389694213867
d_loss_fake:0.0006178886396810412
d_loss_wrong:0.7535851001739502
d_loss:0.6411202319141012
g_loss:0.32512980699539185
Batch:5
d_loss_real:0.9172151684761047
d_loss_fake:0.0007239290862344205
d_loss_wrong:0.7349065542221069
d_loss:0.6425152050651377
g_loss:0.3255003094673157
Batch:6
d_loss_real:0.9067434668540955
d_loss_fake:0.0006495437701232731
d_loss_wrong:0.7141315937042236
d_loss:0.6320670177956345
g_loss:0.3261088728904724
Batch:7
d_loss_real:0.91

d_loss_real:0.911203920841217
d_loss_fake:0.011363688856363297
d_loss_wrong:0.7095561027526855
d_loss:0.6358319083228707
g_loss:0.42156779766082764
Batch:54
d_loss_real:0.9086926579475403
d_loss_fake:0.00893958006054163
d_loss_wrong:0.6892985105514526
d_loss:0.6289058516267687
g_loss:0.43514758348464966
Batch:55
d_loss_real:0.9176110029220581
d_loss_fake:0.015354959294199944
d_loss_wrong:0.6978826522827148
d_loss:0.6371149043552577
g_loss:0.4118709862232208
Batch:56
d_loss_real:0.937019407749176
d_loss_fake:0.017294282093644142
d_loss_wrong:0.6707154512405396
d_loss:0.6405121372081339
g_loss:0.3942197561264038
Batch:57
d_loss_real:0.8898768424987793
d_loss_fake:0.011292858049273491
d_loss_wrong:0.7093392014503479
d_loss:0.625096436124295
g_loss:0.3688235580921173
Batch:58
d_loss_real:0.9127051830291748
d_loss_fake:0.00896555557847023
d_loss_wrong:0.6910379528999329
d_loss:0.6313534686341882
g_loss:0.3580490052700043
Batch:59
d_loss_real:0.9056390523910522
d_loss_fake:0.0091032134369015

d_loss_real:0.8712641000747681
d_loss_fake:0.002251541707664728
d_loss_wrong:0.6864813566207886
d_loss:0.6078152746194974
g_loss:0.3290017247200012
Batch:107
d_loss_real:0.9058549404144287
d_loss_fake:0.0018883259035646915
d_loss_wrong:0.6716508269309998
d_loss:0.6213122584158555
g_loss:0.3281414210796356
Batch:108
d_loss_real:0.875910222530365
d_loss_fake:0.0024689773563295603
d_loss_wrong:0.6925395131111145
d_loss:0.6117072338820435
g_loss:0.3303316831588745
Batch:109
d_loss_real:0.8662557601928711
d_loss_fake:0.0017543314024806023
d_loss_wrong:0.6832529902458191
d_loss:0.6043797105085105
g_loss:0.32772839069366455
Batch:110
d_loss_real:0.8943773508071899
d_loss_fake:0.0021650136914104223
d_loss_wrong:0.6774616241455078
d_loss:0.6170953348628245
g_loss:0.32730960845947266
Batch:111
d_loss_real:0.847943127155304
d_loss_fake:0.0018846007296815515
d_loss_wrong:0.7103762030601501
d_loss:0.6020367645251099
g_loss:0.3278018832206726
Batch:112
d_loss_real:0.8554396033287048
d_loss_fake:0.00

d_loss_real:0.8633173704147339
d_loss_fake:0.0009049372747540474
d_loss_wrong:0.6871017217636108
d_loss:0.6036603499669582
g_loss:0.32608723640441895
Batch:34
d_loss_real:0.8726122975349426
d_loss_fake:0.0008007987635210156
d_loss_wrong:0.6721591949462891
d_loss:0.6045461471949238
g_loss:0.3259345293045044
Batch:35
d_loss_real:0.8602863550186157
d_loss_fake:0.0011480950051918626
d_loss_wrong:0.7055918574333191
d_loss:0.6068281656189356
g_loss:0.32682210206985474
Batch:36
d_loss_real:0.8627626895904541
d_loss_fake:0.0008888356969691813
d_loss_wrong:0.6704885363578796
d_loss:0.5992256878089393
g_loss:0.32773712277412415
Batch:37
d_loss_real:0.8732141256332397
d_loss_fake:0.0017249415395781398
d_loss_wrong:0.685677170753479
d_loss:0.6084575908898842
g_loss:0.32838210463523865
Batch:38
d_loss_real:0.8819789886474609
d_loss_fake:0.001162177650257945
d_loss_wrong:0.68589848279953
d_loss:0.6127546594361775
g_loss:0.32598572969436646
Batch:39
d_loss_real:0.8383022546768188
d_loss_fake:0.001062

KeyboardInterrupt: 